In [8]:
import numpy as np
import os
import cv2
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
import csv
from tensorflow.keras.preprocessing.image import load_img
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [9]:
folder = "chess_positions"
annotations_file = os.path.join(folder, "annotations.csv")

img_dim = 50

piece_to_label = {
    'r': 'black_rook', 'n': 'black_knight', 'b': 'black_bishop',
    'q': 'black_queen', 'k': 'black_king', 'p': 'black_pawn',
    'R': 'white_rook', 'N': 'white_knight', 'B': 'white_bishop',
    'Q': 'white_queen', 'K': 'white_king', 'P': 'white_pawn',
    '1': 'empty'
}

In [10]:
def split_board_to_squares(image_path):
    img = cv2.imread(image_path)
    squares = []
    square_size = img.shape[0] // 8
    for row in range(8):
        for col in range(8):
            square = img[row*square_size:(row+1)*square_size, col*square_size:(col+1)*square_size]
            squares.append(cv2.resize(square, (img_dim, img_dim)))
    return squares

def load_data_and_labels(folder):
    images = []
    labels = []
    with open(annotations_file, 'r', newline='') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            img_path, fen = row
            squares = split_board_to_squares(os.path.join(folder, img_path))
            # conversion vers FEN en étiquettes
            fen_rows = fen.split(' ')[0].split('/')
            assert len(fen_rows) == 8
            for fen_row in fen_rows:
                for char in fen_row:
                    if char.isdigit():
                        for i in range(int(char)):
                            labels.append('empty')
                    else:
                        labels.append(piece_to_label[char])
            images.extend(squares)
    return images, labels

In [ ]:
images, labels = load_data_and_labels(folder)
images = np.array([img_to_array(img) for img in images])
le = LabelEncoder()
labels = le.fit_transform(labels)
labels = to_categorical(labels)

x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [12]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_dim, img_dim, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(le.classes_), activation='softmax')
])

In [13]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=2, validation_data=(x_test, y_test))

Epoch 1/2
1600/1600 [==============================] - 27s 16ms/step - loss: 0.2834 - accuracy: 0.9967 - val_loss: 0.0000e+00 - val_accuracy: 1.0000
Epoch 2/2
1600/1600 [==============================] - 28s 18ms/step - loss: 0.0000e+00 - accuracy: 1.0000 - val_loss: 0.0000e+00 - val_accuracy: 1.0000


In [14]:
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {accuracy:.2f}")

400/400 [==============================] - 2s 6ms/step - loss: 0.0000e+00 - accuracy: 1.0000
Test accuracy: 1.00


In [15]:
model.save('model.h5')

/home/mory/.local/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
